Importing Libraries

In [26]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.figure_factory as ff
import plotly.graph_objects as go
import plotly.express as px
import sqlalchemy
import json

Loading Datasets

In [27]:
# Database Credentials
DB_PARAMS = {
    "dbname": "postgres",
    "user": "dap",
    "password": "dap",
    "host": "localhost",
    "port": "5432"
}

def get_engine():
    conn_str = f"postgresql://{DB_PARAMS['user']}:{DB_PARAMS['password']}@{DB_PARAMS['host']}:{DB_PARAMS['port']}/{DB_PARAMS['dbname']}"
    return sqlalchemy.create_engine(conn_str)

def load_stock_from_sql(table_name, engine):
    query = f"SELECT * FROM {table_name} ORDER BY trade_date ASC"
    df = pd.read_sql(query, engine)
    cols = ['trade_date', 'open_price', 'high_price', 'low_price', 'close_price', 'volume']
    return df[cols]

def load_and_transform_aapl(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)
    df = pd.DataFrame(data)
    column_mapping = {
        "('Close', 'AAPL')": 'close_price',
        "('High', 'AAPL')": 'high_price',
        "('Low', 'AAPL')": 'low_price',
        "('Open', 'AAPL')": 'open_price',
        "('Volume', 'AAPL')": 'volume'
    }
    df = df.rename(columns=column_mapping)
    df.index = pd.to_datetime(df.index.astype(int), unit='ms')
    df.index.name = 'trade_date'
    df = df.reset_index()
    cols = ['trade_date', 'open_price', 'high_price', 'low_price', 'close_price', 'volume']
    return df[cols]

engine = get_engine()
amazon = load_stock_from_sql('stock_data_amzn', engine)
google = load_stock_from_sql('stock_data_googl', engine)
netflix = load_stock_from_sql('stock_data_nflx', engine)
apple = load_and_transform_aapl(r'../../data/raw/stock_data_AAPL.json')

In [28]:
print(apple.shape)
print(google.shape)
print(netflix.shape)
print(amazon.shape)

(2756, 6)
(2756, 6)
(2756, 6)
(2756, 6)


In [29]:
print(apple)
print(google)
print(netflix)
print(amazon)

     trade_date  open_price  high_price   low_price  close_price     volume
0    2015-01-02   24.694231   24.705316   23.798597    24.237547  212818400
1    2015-01-05   24.006990   24.086799   23.368519    23.554739  257142000
2    2015-01-06   23.619031   23.816336   23.195599    23.556957  263188400
3    2015-01-07   23.765348   23.987040   23.654503    23.887280  160423600
4    2015-01-08   24.215383   24.862723   24.097885    24.805082  237458000
...         ...         ...         ...         ...          ...        ...
2751 2025-12-10  277.750000  279.750000  276.440002   278.779999   33038300
2752 2025-12-11  279.100006  279.589996  273.809998   278.029999   33248000
2753 2025-12-12  277.899994  279.220001  276.820007   278.279999   39532900
2754 2025-12-15  280.149994  280.149994  272.839996   274.109985   50409100
2755 2025-12-16  272.820007  275.500000  271.790009   274.609985   37648600

[2756 rows x 6 columns]
      trade_date  open_price  high_price   low_price  close_pri

In [30]:
print(apple.info())
print(google.info())
print(netflix.info())
print(amazon.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2756 entries, 0 to 2755
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   trade_date   2756 non-null   datetime64[ns]
 1   open_price   2756 non-null   float64       
 2   high_price   2756 non-null   float64       
 3   low_price    2756 non-null   float64       
 4   close_price  2756 non-null   float64       
 5   volume       2756 non-null   int64         
dtypes: datetime64[ns](1), float64(4), int64(1)
memory usage: 129.3 KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2756 entries, 0 to 2755
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   trade_date   2756 non-null   object 
 1   open_price   2756 non-null   float64
 2   high_price   2756 non-null   float64
 3   low_price    2756 non-null   float64
 4   close_price  2756 non-null   float64
 5   volume       2756 non

In [31]:
print(apple.columns)
print(google.columns)
print(netflix.columns)
print(amazon.columns)

Index(['trade_date', 'open_price', 'high_price', 'low_price', 'close_price',
       'volume'],
      dtype='object')
Index(['trade_date', 'open_price', 'high_price', 'low_price', 'close_price',
       'volume'],
      dtype='object')
Index(['trade_date', 'open_price', 'high_price', 'low_price', 'close_price',
       'volume'],
      dtype='object')
Index(['trade_date', 'open_price', 'high_price', 'low_price', 'close_price',
       'volume'],
      dtype='object')


In [32]:
# Data is now in standard format: ['trade_date', 'open_price', 'high_price', 'low_price', 'close_price', 'volume']
# Remap to notebook internal names if necessary (the notebook seems to use 'date', 'open', etc. after a loop)
columns_short = ['date', 'open', 'high', 'low', 'close', 'volume']
for df in [apple, google, netflix, amazon]:
    df.columns = columns_short

In [33]:
print(apple.columns)
print(google.columns)
print(netflix.columns)
print(amazon.columns)

Index(['date', 'open', 'high', 'low', 'close', 'volume'], dtype='object')
Index(['date', 'open', 'high', 'low', 'close', 'volume'], dtype='object')
Index(['date', 'open', 'high', 'low', 'close', 'volume'], dtype='object')
Index(['date', 'open', 'high', 'low', 'close', 'volume'], dtype='object')


In [34]:
for df in [apple, google, netflix, amazon]:
    df['date'] = pd.to_datetime(df['date'])
    df[['open', 'high', 'low', 'close', 'volume']] = df[['open', 'high', 'low', 'close', 'volume']].apply(pd.to_numeric)

Preprocessing

In [35]:
for df in [apple, google, netflix, amazon]:
    df['daily_return'] = df['close'].pct_change()
    df['moving_average_200'] = df['close'].rolling(window=200).mean()
    df['moving_average_50'] = df['close'].rolling(window=50).mean()

Visualisation

Stock Closing Prices

In [36]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=apple['date'], y=apple['close'], mode='lines', name='Apple Close Price'))
fig.add_trace(go.Scatter(x=google['date'], y=google['close'], mode='lines', name='Google Close Price'))
fig.add_trace(go.Scatter(x=netflix['date'], y=netflix['close'], mode='lines', name='Netflix Close Price'))
fig.add_trace(go.Scatter(x=amazon['date'], y=amazon['close'], mode='lines', name='Amazon Close Price'))
fig.update_layout(title='Stock Closing Prices Over Time', xaxis_title='Date', yaxis_title='Closing Price (USD)', template='plotly_dark')
fig.show()

50 day moving Averages

In [37]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=apple['date'], y=apple['moving_average_50'], mode='lines', name='Apple MA 50'))
fig.add_trace(go.Scatter(x=google['date'], y=google['moving_average_50'], mode='lines', name='Google MA 50'))
fig.add_trace(go.Scatter(x=netflix['date'], y=netflix['moving_average_50'], mode='lines', name='Netflix MA 50'))
fig.add_trace(go.Scatter(x=amazon['date'], y=amazon['moving_average_50'], mode='lines', name='Amazon MA 50'))
fig.update_layout(title='50-Day Moving Averages', xaxis_title='Date', yaxis_title='Moving Average (USD)', template='plotly_dark')
fig.show()

200 Days Moving Averages

In [38]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=apple['date'], y=apple['moving_average_200'], mode='lines', name='Apple MA 200'))
fig.add_trace(go.Scatter(x=google['date'], y=google['moving_average_200'], mode='lines', name='Google MA 200'))
fig.add_trace(go.Scatter(x=netflix['date'], y=netflix['moving_average_200'], mode='lines', name='Netflix MA 200'))
fig.add_trace(go.Scatter(x=amazon['date'], y=amazon['moving_average_200'], mode='lines', name='Amazon MA 200'))
fig.update_layout(title='200-Day Moving Averages', xaxis_title='Date', yaxis_title='Moving Average (USD)', template='plotly_dark')
fig.show()

Correlation

In [39]:
returns_df = pd.DataFrame({
    'Apple': apple['daily_return'],
    'Google': google['daily_return'],
    'Netflix': netflix['daily_return'],
    'Amazon': amazon['daily_return']
})
corr_matrix = returns_df.corr()
fig = ff.create_annotated_heatmap(
    z=corr_matrix.values,
    x=list(corr_matrix.columns),
    y=list(corr_matrix.index),
    colorscale='Viridis',
    showscale=True
)
fig.update_layout(title='Correlation Heatmap of Daily Returns', template='plotly_dark')
fig.show()

Trading Volume Analysis

In [40]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=apple['date'], y=apple['volume'], mode='lines', name='Apple Volume'))
fig.add_trace(go.Scatter(x=google['date'], y=google['volume'], mode='lines', name='Google Volume'))
fig.add_trace(go.Scatter(x=netflix['date'], y=netflix['volume'], mode='lines', name='Netflix Volume'))
fig.add_trace(go.Scatter(x=amazon['date'], y=amazon['volume'], mode='lines', name='Amazon Volume'))
fig.update_layout(title='Trading Volume Over Time', xaxis_title='Date', yaxis_title='Volume', template='plotly_dark')
fig.show()

Volatility Analysis

In [41]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=apple['date'], y=apple['high'] - apple['low'], mode='lines', name='Apple Volatility'))
fig.add_trace(go.Scatter(x=google['date'], y=google['high'] - google['low'], mode='lines', name='Google Volatility'))
fig.add_trace(go.Scatter(x=netflix['date'], y=netflix['high'] - netflix['low'], mode='lines', name='Netflix Volatility'))
fig.add_trace(go.Scatter(x=amazon['date'], y=amazon['high'] - amazon['low'], mode='lines', name='Amazon Volatility'))
fig.update_layout(title='Daily Volatility Over Time', xaxis_title='Date', yaxis_title='Volatility (USD)', template='plotly_dark')
fig.show()